# Week 4 - Probabilistic Programming with STAN

## Part 1 - Simple STAN model

Hello again. We've finally arrived to STAN - a platform for probabilistic programming!



...the usual imports...

**well, notice the new packages!!**

In [0]:
# First, we need to download an auxiliary Python file for STAN
!wget http://mlsm.man.dtu.dk/mbml/pystan_utils.py

--2020-02-17 14:02:33--  http://mlsm.man.dtu.dk/mbml/pystan_utils.py
Resolving mlsm.man.dtu.dk (mlsm.man.dtu.dk)... 192.38.87.226
Connecting to mlsm.man.dtu.dk (mlsm.man.dtu.dk)|192.38.87.226|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2661 (2.6K) [text/x-python]
Saving to: ‘pystan_utils.py’

pystan_utils.py     100%[===================>]   2.60K  --.-KB/s    in 0s      

2020-02-17 14:02:34 (437 MB/s) - ‘pystan_utils.py’ saved [2661/2661]



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pystan
import pystan_utils # our own small package with utilities to use with STAN

# matplotlib options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

In [0]:
# usually have to run this twice to take effect
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

### 1.1. Hello World and variations

Every respectable programming language requires that newcomers start with a small "Hello World" code. Besides being a familiar initial exercise, it also lets you learn a basic tool: how to print something! 

Don't forget. It may be useful for you later, for example for debugging!

We'll do the program for you



In [0]:
Hello_STAN="""
transformed data {
    print("Hello World from STAN");
}
"""

# compile model
sm = pystan.StanModel(model_code=Hello_STAN)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_525af6d7864d89b75150c90990c74893 NOW.


To run the program above in pystan, we need to call the following method. Notice the parameters.

In [0]:
# run inference
fit = sm.sampling(algorithm="Fixed_param", iter=10)

Look carefully at your STAN output. It is NOT in this notebook... you'll have to go to the console to see it! 

Can you control the number of times the sentence "Hello World..." shows up? 

Can you try to print from other STAN code blocks? What are the effects?

The next step is to exchange data TO/FROM your STAN and Python codes, with a trivial model. 

The example below sends one real number to STAN, as data point a, then it gets it sends it back through two values: a single sample, x, from a normal distribution (centered at a); a new variable, b, that simply equals a.

In [0]:
tofrom_STAN="""
data {
    real a;
}
parameters {
    real x;
}
model {  
    x ~ normal(a, 10);
}
generated quantities {
    real b=a;
}
"""

# compile model
sm = pystan.StanModel(model_code=tofrom_STAN)

This time, we will use HMC (Hamiltonian Monte Carlo)

In [0]:
# run inference
fit = sm.sampling(data={'a':5}, algorithm="HMC", seed=0, iter=1000)

Take a look at the results (variable fit). 

Actually, there's still more to it. In fact, STAN didn't just draw a single value x as you can see from the table (look at n_eff, it obtained 2000 draws!). All the 2000 samples are available through fit.extract()

Make a histogram for x. What kind of distribution would you expect? ;-)

What's happening is that each single iteration (of 1000) of each single chain (of 4) is drawing one sample. Since STAN only uses data after the first 50% iterations, it becomes 500x4=2000 samples of the same variable. That is why x is not a single value, but instead a distribution. 

Actually you could have used 
>fit.traceplot('x')

Let's ellaborate a bit more with the input/outputs.

In [0]:
simple_STAN="""
data {
    real a;
}
transformed data {
    real b=2*a;
    int c=10;
    print("c=", c, " b=", b); // check this in the console!!!
}
parameters {
    real x;
}
model {
    x ~ normal(a, 10);
}
generated quantities {
    real d=a+c;
    real e=4.0;
}
"""

# compile model
sm = pystan.StanModel(model_code=simple_STAN)

Let's run it

In [0]:
# run inference
fit = sm.sampling(data={'a':5}, algorithm="HMC", seed=0, iter=1000)

Notice that the function call returns a value into the 'fit' variable. Do you want to take a look at it? Can you retrieve the value of the parameter 'b'? What about 'd'?

With one data point at a time, we won't go that far, so it's time to work with vectors...  

In [0]:
simple_vector_STAN="""
data {
    int N;
    vector[N] a;
}
transformed data {
    real b=mean(a);
    print("b=", b); // check this in the console!!!
}
parameters {
    real x;
    real y;
}
model {
    x ~ normal(b, 10);
    a ~ normal(y, 10);
}
"""

# compile model
sm = pystan.StanModel(model_code=simple_vector_STAN)

In [0]:
a = np.array([1.0, 3.1, 3.4, 6.7, 4.0, 0.0, 1.1, 2.0])

# run inference
fit = sm.sampling(data={'N':len(a), 'a':a}, algorithm="HMC", seed=0, iter=1000)

Try to do a traceplot. Can you interpret the results?

## 2 Your first STAN model


### 2.1 The cyclist problem

Let's do the cyclist problem from the lecture slides: We have a dataset of observations (travel times of bicycle trips), and we want to estimate its distribution, assuming it follows a Gaussian curve of some form. 



As detailed in the lecture slides, you can code your STAN model directly into a Python string


In [0]:
cyclist_STAN="""
data {
    int<lower=0> N; // number of samples
    vector[N] tt;   // observed travel times
}
parameters {
    real at;          // average travel time
    real<lower=0> tu; // traffic uncertainty
}
model {
    at ~ normal(12, 10);
    tu ~ cauchy(0, 10);
    tt ~ normal(at, tu);
}
"""

# compile model
sm = pystan.StanModel(model_code=cyclist_STAN)

It is always good to have an intuitive notion of the prior forms (e.g. are they "too" informative, maybe? Or too wide?)

Can you plot the priors for _at_ and _tu_?

Given what we know of the problem, do they make sense? 




Below is the Python code that calls for the STAN code above. 

In [0]:
# store data in python dictionary
cyclist_dat = {'N': 14,
               'tt': [13,17,16,32,12,13,28,12,14,18,36,16,16,31]}

# run inference
fit = sm.sampling(data=cyclist_dat, algorithm="NUTS", seed=0, iter=1000)

# show results
print(fit)

# extract samples
samples = fit.extract(permuted=True)

The resulting MCMC samples that you get are in the "samples" object that we created above to collect all fit.extract() data. Try to **inspect** it, particularly do a histogram for the "at" and "tu" variables.

Check how many samples you have, and relate the number with the statistics that STAN provides above:
- How many samples did it actually generate?
- How many is it using for the final statistics?
- How many chains is it using (and how many samples being used in the end by each chain?)



To add to STAN's plot that you've used already, and we put some standard outout code here for you. ;-)

In [0]:
from scipy import stats

print("\nAverate travel time:")
print("posterior mode=%.2f mean=%.2f std=%.2f" % (stats.mode(samples["at"].round(0))[0][0], samples['at'].mean(), samples['at'].std()))

print("\nTraffic uncertainty:")
print("posterior mode=%.2f mean=%.2f std=%.2f" % (stats.mode(samples["tu"].round(1))[0][0], samples['tu'].mean(), samples['tu'].std()))

print ("\nLog marginal likelihood:", samples["lp__"].mean())

fit.plot()
plt.show()

**note:** the concept of "Log marginal likelihood" corresponds to the marginal probability of all data (denominator in Bayes formula)



Redefine the priors above and re-estimate the model (try only a couple of extreme values, just to see the effect)

### 2.2 Obtaining useful results from your model

Using the generated MCMC samples, you can find some useful answers.  For example, can you calculate the probability that the next trip takes less than 18 minutes?

Notice that the certainty of your answer depends on the number of samples... 

Alternatively, we can use the "generated quantities" block of the STAN program to compute this probability. Lets extend our previous STAN program with a "generated quantities" block to compute the probability that the next trip takes less than 18 minutes.

In [0]:
cyclist_STAN="""
data {
    int<lower=0> N; // number of samples
    vector[N] tt;   // observed travel times
}
parameters {
    real at;          // average travel time
    real<lower=0> tu; // traffic uncertainty
}
model {
    at ~ normal(12, 10);
    tu ~ cauchy(0, 10);
    tt ~ normal(at, tu);
}
generated quantities {
    real prediction;
    int<lower=0,upper=1> is_less;    
    
    prediction = normal_rng(at, tu); // make prediction
    if (prediction < 18)             // check if prediction is less then 18
        is_less = 1;
    else
        is_less = 0;
}
"""

# compile model
sm = pystan.StanModel(model_code=cyclist_STAN)

Analyse the code above carefully. 

Notice how the "generated quantities" follows closely the generative process to sample observed travel times (predictions).

Run inference given some observed data:

In [0]:
# store data in python dictionary
cyclist_dat = {'N': 14,
               'tt': [13,17,16,32,12,13,28,12,14,18,36,16,16,31]}

# run inference
fit = sm.sampling(data=cyclist_dat, algorithm="NUTS", seed=0, iter=1000)

# show results
print(fit)

Did you get a similar result?

### 2.3 Revised cyclist model: mixture model

We will now consider a more realistic model. Since sometimes cyclists are prone to extraordinary circumstances (e.g. flat tire, forgot something at home, ran into a friend and started chatting, etc.), the distribution of travel times can be bimodal, such that on certain days ("abnormal" days) the distribution of travel times is radically different the distribution of "normal" days. 

We will formulate this assumption as mixture of two Gaussians. 

Can you write the new (revised) model in STAN?

Hints:
- We will now have two Gaussians instead of one;
- We need a mixture parameter $\pi$ (real parameter between 0 and 1) that controls the mixing proportions of the two Gaussians (see lecture slides);
- The likelihood is now of the form: 

$\pi \, \mathcal{N}(at_o,tu_o) + (1-\pi) \, \mathcal{N}(at_a,tu_a)$

We can encode the likelihood in STAN using the function "log_mix()" as follows:

```python
for (n in 1:N)
    target += log_mix(pi, normal_log(tt[n], at_o, tu_o), normal_log(tt[n], at_a, tu_a));
```

The ```target``` variable is simply the log joint probability of the model. This is essentially what STAN needs to know about the model in order to perform inference. Therefore, if you write ```at ~ normal(10, 10)``` in the model block of your STAN program, that is actually equivalent to writting:

```python
for (n in 1:N)
    target += normal_log(at, 10, 10);
```

The former style is much more intuitive. However, in some cases (like this one), we need to directly increment the ```target``` quantity in order to exploit more advanced functionality of STAN. 

In [0]:
cyclist_STAN_mixture = """
data {
    int<lower=0> N; // number of samples
    vector[N] tt;   // observed travel times
}
parameters {
    // YOUR CODE HERE
}
model {
    // YOUR CODE HERE
}
"""

# compile model
sm = pystan.StanModel(model_code=cyclist_STAN_mixture)

Lets run inference on the revised model using the same cyclist data from before:

In [0]:
# store data in python dictionary
cyclist_dat = {'N': 14,
               'tt': [13,17,16,32,12,13,28,12,14,18,36,16,16,31]}

# run inference
fit = sm.sampling(data=cyclist_dat, algorithm="NUTS", seed=0, chains=1, iter=1000)

# show results
print(fit)

# extract samples
samples = fit.extract(permuted=True)

Look at the results. What can you tell?

### 2.4 Obtaining useful results from your revised model

As you did for the previous model with a single Gaussian, can you compute the probability that the next trip takes less than 18 minutes (given the new and more realistic model)?

In [0]:
cyclist_STAN_mixture = """
data {
    int<lower=0> N; // number of samples
    vector[N] tt;   // observed travel times
}
parameters {
    // SAME CODE AS YOU WROTE BEFORE IN 2.3
}
model {
    // SAME CODE AS YOU WROTE BEFORE IN 2.3
}
generated quantities {
    // WRITE YOUR NEW CODE HERE
}
"""

# compile model
sm = pystan.StanModel(model_code=cyclist_STAN_mixture)

Lets run inference on the model and check our answer:

In [0]:
# store data in python dictionary
cyclist_dat = {'N': 14,
               'tt': [13,17,16,32,12,13,28,12,14,18,36,16,16,31]}

# run inference
fit = sm.sampling(data=cyclist_dat, algorithm="NUTS", seed=0, iter=1000)

# show results
print(fit)

Did you notice how different the answer when compared to previous model (single Gaussian)? Which one makes more sense to you? 

## Part 2 -  Mixture model

We know that by know you think you've had enough about Mixture Models... but in fact, there's (even) much more to it! :-)

For example, your well-known K-Means clustering algorithm is in fact a special case of a mixture model. 

Let's create a dataset with 200 samples that are distributed around 2 centers with 0.6 standard deviation. We will use the function [`make_blobs` ](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html)that generates isotropic Gaussian blobs for clustering.

In [0]:
# Generate some data
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=100, centers=2, cluster_std=0.60, random_state=0)
X = X[:, ::-1] # flip axes for better plotting

In [0]:
# Plot the data with K Means Labels
from sklearn.cluster import KMeans
kmeans = KMeans(2, random_state=0)
labels = kmeans.fit(X).predict(X)
sns.scatterplot(X[:, 0], X[:, 1], hue=labels)
plt.title('Dataset with k-means labels');

Let's look at their centers

In [0]:
kmeans.cluster_centers_

It's time for our STAN GMM model. Get ready...

Study the code carefully

In [0]:
GMM_STAN="""
data {
    int<lower=0> D;  // number of dimensions
    int<lower=0> N;  // number of samples
    vector[D] X[N] ; // observed points
    int<lower=0> K;  //number of clusters
}

parameters {
    simplex[K] theta;             // mixing proportions
    vector[D] mu[K];              // mixture component means
    vector<lower=0>[D] sigma[K];  // covariance matrices
}
model {
    vector[K] log_theta = log(theta);  // cache log calculation
    
    for (k in 1:K){
        mu[k] ~ normal(0, 10);
        sigma[k] ~ cauchy(0, 10);
    }
    
    for (n in 1:N) {
        vector[K] lps = log_theta;
        for (k in 1:K)
            lps[k] += normal_lpdf(X[n] | mu[k], sigma[k]);
        target += log_sum_exp(lps);
    }
}
"""

# compile model
sm = pystan.StanModel(model_code=GMM_STAN)

Inspect carefully the following code too. Particularly, notice that we're running with a single chain and 4000 iterations, instead of the default (chains=4, iterations=1000). Why would that be? 

To answer this question, run this code and check the results (compare with the centers above).

In [0]:
K=2
N=len(X)
D=2

data={'X':X, 'N': N, 'K':K, 'D':2}

# run inference
fit = sm.sampling(data=data, algorithm="NUTS", chains=1, seed=0, iter=4000)

# show results
print(fit)

Now, try to put the default values (chains=4, iterations=1000). What's the problem? 


Create and test a new version of the code above with higher K (e.g. K=3). 

### Bonus: Alternative way of doing inference using ADVI

In [0]:
fit = sm.vb(data=data, iter=10000, algorithm="fullrank", grad_samples=10, seed=42, verbose=True)

Did you notice how much faster that was? Did it get the right result? Let's see:

In [0]:
mus = pystan_utils.vb_extract_variable(fit, "mu", var_type="matrix", dims=[2,2])
print("mus:", mus)

mus: [[1.01784005 2.05098601]
 [4.330718   0.95910048]]
